In [1]:
:l Queue.hs

In [4]:
--BankersQueue
module BankersQueue (BankersQueue) where

import Prelude hiding (head, tail)
import Queue

data BankersQueue a = BQ Int [a] Int [a]

check lenf f lenr r = 
    if lenf <= lenr then BQ lenf f lenr r else BQ (lenf + lenr) (f ++ reverse r) 0 []
    
instance Queue BankersQueue where
    empty = BQ 0 [] 0 []
    isEmpty (BQ lenf _ _ _) = lenf == 0
    
    snoc (BQ lenf f lenr r) x = check lenf f (lenr + 1) (x : r)
    
    head (BQ lenf [] lenr r) = error "Error: Empty Queue"
    head (BQ lenf (x : f') lenr r) = x
    
    tail (BQ lenf [] lenr r) = error "Error: Empty Queue"
    tail (BQ lenf (x : f') lenr r) = check (lenf - 1) f' lenr r

In [5]:
:l Heap.hs

In [10]:
--Exercise 6.5
module SizedHeap where

import qualified Heap as H

data SizedHeap h a = SH !Int (h a) deriving (Show)

instance H.Heap h => H.Heap (SizedHeap h) where
    empty = SH 0 H.empty
    
    isEmpty (SH s h) = s == 0 
    
    insert x (SH s h) = SH (s+1) (H.insert x h)
    
    merge (SH s1 h1) (SH s2 h2) = SH (s1+s2) (H.merge h1 h2)
    
    findMin (SH s h) = H.findMin h
    
    deleteMin (SH s h) = SH (s-1) (H.deleteMin h)

In [11]:
--Exercise 6.5
:l BinomialHeap.hs
:l SizedHeap.hs

import Heap
import BinomialHeap
import SizedHeap

main :: IO ()
main = do
    print $ (foldr insert empty [1..8] :: SizedHeap BHeap Int)
    
main

SH 8 (BH [Node {_rank = 3, _value = 1, _children = [Node {_rank = 2, _value = 5, _children = [Node {_rank = 1, _value = 7, _children = [Node {_rank = 0, _value = 8, _children = []}]},Node {_rank = 0, _value = 6, _children = []}]},Node {_rank = 1, _value = 3, _children = [Node {_rank = 0, _value = 4, _children = []}]},Node {_rank = 0, _value = 2, _children = []}]}])

In [18]:
--PhysicistsQueue
:l Queue.hs

module PhysicistsQueue (PhysiciststQueue) where

import Prelude hiding (head, tail)
import Queue

data PhysicistsQueue a = PQ [a] Int [a] Int [a]

check w lenf f lenr r = if lenr <= lenf 
    then check w lenf f lenr r
    else checkw f (lenf+lenr) (f ++ reverse r) 0 []
    
checkw [] lenf f lenr r = PQ f lenf f lenr r
checkw w lenf f lenr r = PQ w lenf f lenr r

instance Queue PhysicistsQueue where 
    empty = PQ [] 0 [] 0 []
    isEmpty (PQ w lenf f lenr r) = lenf == 0
    
    snoc (PQ w lenf f lenr r) x = check w lenf f (lenr+1) (x : r)
    
    head (PQ [] lenf f lenr r) = error "Error: Empty Queue"
    head (PQ (x : w) lenf f lenr r) = x
    
    tail (PQ [] lenf f lenr r) = error "Error: Empty Queue"
    tail (PQ (x : w) lenf f lenr r) = check w (lenf-1) (Prelude.tail f) lenr r

: 

In [19]:
--Sortable Collections
module Sortable (Sortable(..)) where
    class Sortable s where
        empty :: Ord a => s a
        add :: Ord a => a -> s a -> s a
        sort :: Ord a => s a -> [a]

In [20]:
:l Sortable.hs

In [21]:
module BottomUpMergeSort (MergeSort) where
    import Sortable
    
    data MergeSort a = MS Int [[a]]
    
    mrg [] ys = ys
    mrg xs [] = xs
    mrg xs@(x : xs') ys@(y : ys') = 
        if x <= y then x : mrg xs' ys else y : mrg xs ys'
        
    instance Sortable MergeSort where
        empty = MS 0 []
        
        add x (MS size segs) = MS (size+1) (addSeg [x] segs size)
            where addSeg seg segs size = 
                            if size `mod` 2 == 0 then seg : segs
                            else addSeg (mrg seg (head segs)) (tail segs) (size `div` 2)
                            
        sort (MS size segs) = foldl mrg [] segs

In [24]:
--Exercise 6.7 (b)
:l BottomUpMergeSort.hs

import BottomUpMergeSort

extractSmallest :: Ord a => Int -> MergeSort a -> [a]
extractSmallest k = take k . sort 